In [11]:
import nltk
from nltk.corpus import stopwords
import re
from gensim import corpora
import joblib
import pandas as pd

# Tải mô hình
tfidf = joblib.load("tfidf_model.pkl")
kmeans = joblib.load("kmeans_model.pkl")
lda_model = joblib.load("lda_model.pkl")
dictionary = joblib.load("dictionary.pkl")
svd = joblib.load("svd_model.pkl")

In [12]:
# Hàm làm sạch tweet
stop_words = set(stopwords.words("english"))
def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Xóa URL
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Xóa ký tự đặc biệt
    text = text.lower()
    return " ".join([word for word in text.split() if word not in stop_words])

# Hàm dự đoán sự quan tâm
def predict_user_interest(new_tweets):
    # Làm sạch tweet
    cleaned_tweets = [clean_text(tweet) for tweet in new_tweets]
    
    # Biểu diễn TF-IDF
    tfidf_new = tfidf.transform(cleaned_tweets)
    
    # Giảm chiều dữ liệu mới
    tfidf_new_reduced = svd.transform(tfidf_new)  # Giảm từ 1000 xuống 50 đặc trưng
    
    # Dự đoán cụm (K-Means)
    cluster = kmeans.predict(tfidf_new_reduced)[0]
    
    # Chuẩn bị dữ liệu cho LDA
    tokenized_tweets = [tweet.split() for tweet in cleaned_tweets]
    bow_corpus = [dictionary.doc2bow(text) for text in tokenized_tweets]
    
    # Dự đoán chủ đề (LDA)
    topic_dist = lda_model[bow_corpus[0]]  # Phân phối chủ đề cho tweet đầu tiên
    main_topic = max(topic_dist, key=lambda x: x[1])[0]  # Chủ đề có xác suất cao nhất
    
    # Tính trung bình TF-IDF để diễn giải
    tfidf_df = pd.DataFrame(tfidf_new.toarray(), columns=tfidf.get_feature_names_out())
    user_tfidf_new = tfidf_df.mean(axis=0)
    top_keywords = user_tfidf_new.sort_values(ascending=False).head(5)
    
    return {
        "cluster": int(cluster),
        "main_topic": int(main_topic),
        "top_keywords": top_keywords.to_dict()
    }

In [13]:
new_tweets = ["I love listening to music and going to concerts", "Work is so stressful today"]
result = predict_user_interest(new_tweets)
print("Dự đoán sự quan tâm:")
print(f"Cụm: {result['cluster']}")
print(f"Chủ đề quan tâm: {result['main_topic']}")
print("Top từ khóa:", result["top_keywords"])

Dự đoán sự quan tâm:
Cụm: 3
Chủ đề quan tâm: 4
Top từ khóa: {'work': 0.35982173753965196, 'today': 0.3471718842215564, 'listening': 0.29801750284411727, 'music': 0.28849011241882017, 'going': 0.2027887122945104}
